# Productionize Script

### Goals

1. Find services with Volume mounts, create new images with code packaged in them
2. Add secrets.entrypoint.sh to every service that has environment variables.
3. Separate the services into n separate docker files


In [52]:
import os
import subprocess
import time
import yaml
import re
import json
import docker
client = docker.from_env()

In [54]:
# Generate version number for built
version = str(int(time.time()))
version

'1523028430'

In [57]:
user_name = config['user_name']

In [69]:
#Set input file
input_file = config['compose_file']

In [248]:
# Load the services from the input docker-compose.yml file.
stack = yaml.load(open(input_file))

In [238]:
stack["services"]

{'data_store': {'image': 'redis:alpine'},
 'db-data': {'container_name': 'mongodb_02',
  'environment': ['MONGO_INITDB_ROOT_USERNAME=${MONGO_INITDB_ROOT_USERNAME}',
   'MONGO_INITDB_ROOT_PASSWORD=${MONGO_INITDB_ROOT_PASSWORD}'],
  'image': 'mongo:3.6.0',
  'ports': ['27018:27018'],
  'restart': 'always',
  'volumes': ['./data-producers/db-data:/data/db']},
 'iex': {'command': "jupyter-notebook --no-browser --port 7777 --ip='*' --allow-root",
  'container_name': 'iex_02',
  'depends_on': ['data_store'],
  'environment': ['MONGODB_USER=${MONGO_INITDB_ROOT_USERNAME}',
   'MONGODB_PASS=${MONGO_INITDB_ROOT_PASSWORD}'],
  'image': 'madderle/scaling-docker:data-producer-1.0',
  'ports': ['7777:7777'],
  'volumes': ['./data-producers/IEX:/home/ds']},
 'twitter': {'command': "jupyter-notebook --no-browser --port 9999 --ip='*' --allow-root",
  'container_name': 'twitter_02',
  'depends_on': ['data_store'],
  'environment': ['MONGODB_USER=${MONGO_INITDB_ROOT_USERNAME}',
   'MONGODB_PASS=${MONGO_I

In [239]:
stack['services']['twitter']['volumes'][0]

'./data-producers/Twitter:/home/ds'

In [53]:
################ Configuration ########################

# Get Project Name
project_name = os.path.basename(os.path.realpath(".")).lower()

# Read JSON File
config = json.load(open('production_configuration.json'))

#Set input file
input_file = config['compose_file']

# Load the services from the input docker-compose.yml file.
stack = yaml.load(open(input_file))


'scaling-docker'

In [245]:
# Iterate over all the services that have a volume
# Need to figure out a way to add code. This can be done in teh dockerfile. Maybe need to have 2 Dockerfiles. 1 for Development and 1 for production. 

#This function need to include multiple things:
# FROM image name
# Add Volume Data
# COPY Entry Point script if have environment variables
# Set entry point script if have environment variables

def build_volume(input):
    output = ''
    #print(input)
    if input:
        source = input.split(":")[0]
        destination = input.split(":")[1]
        output = "\nADD " + source + ' ' + destination
    
    return output

def build_environment(environment):
    output = ''
    if environment:
        copy = '\nCOPY ./secrets-entrypoint.sh /secrets-entrypoint.sh'

        entry = '\nENTRYPOINT ["/secrets-entrypoint.sh"]'
        
    return copy+entry

def build_docker_file(filename, image_name, volume, environment):
    file = open(filename,"w")
    file.write("FROM " + image_name )
    file.write("\n" + volume)
    file.write("\n"+ environment)
    file.close()

In [223]:
volume = stack['services']['twitter']['volumes'][0]
# volume_check = build_volume(volume)
# volume_check
volume

'./data-producers/Twitter:/home/ds'

In [249]:
##################### Build Docker Images ##########################################

services = stack["services"]
version = str(int(time.time()))
# Find out the services that need to be rebuilt and create docker files. 

build_operation = {}


for service_name, service in services.items():
    build = False
    volume_file_output=''
    environment=''
    
    # Check to see if have volume
    if "volumes" in service and 'db' not in service_name:
        volume = service['volumes'][0]
        
        source = volume.split(":")[0]
        destination = volume.split(":")[1]
        volume_file_output = "\nADD " + source + ' ' + destination
        build = True
        del service['volumes']
    
    # Check to see if have environment
    if "environment" in service:
        environment = build_environment(True)
        build = True
        del service['environment']
    
    if build==True:
        image_name = service['image']
        folder = './Production_Images/{}'.format(service_name)
        # Make folder if dont exist
        if not os.path.exists(folder):
            os.makedirs(folder)
        filename = 'Dockerfile'
        full_path = folder + '/' + filename
        
        # Create Docker files
        print('Building Docker file for {}...'.format(service_name))
        build_docker_file(full_path, image_name, volume_file_output, environment)
        hub_image = "{}/{}:{}-production-{}".format(user_name, project_name, service_name, version)
        
        # Code to build the image
        print('Building {}...'.format(hub_image))
        client.images.build(path='.', tag=hub_image, dockerfile=full_path)
        print('Built Successfully!')
                   
        # Code to push the image to dockerhub
        print('Pushing {}...'.format(hub_image))
        client.images.push(repository='{}/{}'.format(user_name, project_name), tag='{}-production-{}'.format(service_name, version))
        print('Pushed Successfully!')
                
        # Update the image
        service['image'] = hub_image

Building Docker file for twitter...
Building madderle/scaling-docker:twitter-production-1523046012...
Built Successfully!
Pushing madderle/scaling-docker:twitter-production-1523046012...
Pushed Successfully!
Building Docker file for iex...
Building madderle/scaling-docker:iex-production-1523046012...
Built Successfully!
Pushing madderle/scaling-docker:iex-production-1523046012...
Pushed Successfully!
Building Docker file for db-data...
Building madderle/scaling-docker:db-data-production-1523046012...
Built Successfully!
Pushing madderle/scaling-docker:db-data-production-1523046012...
Pushed Successfully!


In [250]:
# Output to separate compose files based on CSV of which services should be together
# Have to copy the stack into a temporary variable and then remove the services  not included
# Then output stack to a file, and then clean up the fil to make it not buggy.
services

{'data_store': {'image': 'redis:alpine'},
 'db-data': {'container_name': 'mongodb_02',
  'image': 'madderle/scaling-docker:db-data-production-1523046012',
  'ports': ['27018:27018'],
  'restart': 'always',
  'volumes': ['./data-producers/db-data:/data/db']},
 'iex': {'command': "jupyter-notebook --no-browser --port 7777 --ip='*' --allow-root",
  'container_name': 'iex_02',
  'depends_on': ['data_store'],
  'image': 'madderle/scaling-docker:iex-production-1523046012',
  'ports': ['7777:7777']},
 'twitter': {'command': "jupyter-notebook --no-browser --port 9999 --ip='*' --allow-root",
  'container_name': 'twitter_02',
  'depends_on': ['data_store'],
  'image': 'madderle/scaling-docker:twitter-production-1523046012',
  'ports': ['9999:9999']}}

In [156]:
# Reset the stack services to the new updated ones
stack = yaml.load(open(input_file))
stack['services'] = services

In [104]:
type(config['services'])

list

In [105]:
config['services']

[{'data_producers': ['twitter', 'iex'],
  'database': ['db-data'],
  'redis': ['data_store']}]

In [157]:
stack

{'services': {'data_store': {'image': 'redis:alpine'},
  'db-data': {'container_name': 'mongodb_02',
   'environment': ['MONGO_INITDB_ROOT_USERNAME=${MONGO_INITDB_ROOT_USERNAME}',
    'MONGO_INITDB_ROOT_PASSWORD=${MONGO_INITDB_ROOT_PASSWORD}'],
   'image': 'mongo:3.6.0',
   'ports': ['27018:27018'],
   'restart': 'always',
   'volumes': ['./data-producers/db-data:/data/db']},
  'iex': {'command': "jupyter-notebook --no-browser --port 7777 --ip='*' --allow-root",
   'container_name': 'iex_02',
   'depends_on': ['data_store'],
   'image': 'madderle/scaling-docker:iex-production-1523028430',
   'ports': ['7777:7777']},
  'twitter': {'command': "jupyter-notebook --no-browser --port 9999 --ip='*' --allow-root",
   'container_name': 'twitter_02',
   'depends_on': ['data_store'],
   'image': 'madderle/scaling-docker:twitter-production-1523028430',
   'ports': ['9999:9999']}},
 'version': '2'}

In [165]:
stack['services']['twitter']

{'command': "jupyter-notebook --no-browser --port 9999 --ip='*' --allow-root",
 'container_name': 'twitter_02',
 'depends_on': ['data_store'],
 'image': 'madderle/scaling-docker:twitter-production-1523028430',
 'ports': ['9999:9999']}

In [166]:
# Goal want to remove all services from stack not found in the items list
# Reset the stack services to the new updated ones
# stack = yaml.load(open(input_file))
# stack['services'] = services

for service_list in config['services']:
    for key, value in service_list.items():
        temp_stack = {}
        temp_stack['services'] = {}
        temp_stack['version'] = stack['version']
        
        # Add services
        for service in value:
            temp_stack['services'][service] = stack['services'][service]
           # Check depend on. If not in list, then need to remove
            try:
                if stack['services'][service]['depends_on'] not in value:
                    del temp_stack['services'][service]['depends_on']
            except:
                pass
                    
        # output yaml file using key in filename
        output_file = '{}_docker_compose.yml'.format(key)
        print('Creating docker file: {}'.format(output_file))
        with open(output_file, "w") as f:
            yaml.safe_dump(temp_stack, f, default_flow_style=False)
        # yaml that is produced is a bit buggy.
        fh = open(output_file, "r+")
        lines = map(lambda a: re.sub(r"^\s{4}-", "      -", a), fh.readlines())
        fh.close()
        with open(output_file, "w") as f:
            f.writelines(lines)
        print('File Built Successfully')


Creating docker file: data_producers_docker_compose.yml
File Built Successfully
Creating docker file: redis_docker_compose.yml
File Built Successfully
Creating docker file: database_docker_compose.yml
File Built Successfully


In [163]:
temp_stack

{'services': {'db-data': {'container_name': 'mongodb_02',
   'environment': ['MONGO_INITDB_ROOT_USERNAME=${MONGO_INITDB_ROOT_USERNAME}',
    'MONGO_INITDB_ROOT_PASSWORD=${MONGO_INITDB_ROOT_PASSWORD}'],
   'image': 'mongo:3.6.0',
   'ports': ['27018:27018'],
   'restart': 'always',
   'volumes': ['./data-producers/db-data:/data/db']}},
 'version': '2'}

## Real Script

In [ ]:
########################## Imports ################################
import os
import subprocess
import time
import yaml
import re
import json
import docker
client = docker.from_env()

In [ ]:
################ Configuration ########################

# Get Project Name
project_name = os.path.basename(os.path.realpath(".")).lower()

# Read JSON File
config = json.load(open('production_configuration.json'))

#Set input file
input_file = config['compose_file']

# Load the services from the input docker-compose.yml file.
stack = yaml.load(open(input_file))

In [ ]:
##################### Build Docker Images #################

def build_docker_file(filename, image_name, source, destination):
    file = open(filename,"w")
    file.write("FROM " + image_name )
    file.write("\nADD " + source + ' ' + destination)
    file.close()
    
services = stack["services"]
version = str(int(time.time()))
# Find out the services that need to be rebuilt and create docker files. 

for service_name, service in services.items():
    if "volumes" in service and 'db' not in service_name:
        volume = service['volumes'][0]
        source = volume.split(":")[0]
        destination = volume.split(":")[1]
        image_name = service['image']
        folder = './Production_Images/{}'.format(service_name)
        # Make folder if dont exist
        if not os.path.exists(folder):
            os.makedirs(folder)
        filename = 'Dockerfile'
        full_path = folder + '/' + filename
        
        # Create Docker files
        build_docker_file(full_path, image_name, source, destination)
        hub_image = "{}/{}:{}-production-{}".format(user_name, project_name, service_name, version)
        
        # Code to build the image
        print('Building {}...'.format(hub_image))
        client.images.build(path='.', tag=hub_image, dockerfile=full_path)
        print('Built Successfully!')
                   
        # Code to push the image to dockerhub
        print('Pushing {}...'.format(hub_image))
        client.images.push(repository='{}/{}'.format(user_name, project_name), tag='{}-production-{}'.format(service_name, version))
        print('Pushed Successfully!')
        # Delete the environment variables
        try:
            del service['environment']
        except:
            pass
        
        
        # Delete the volumes
        del service['volumes']
        # Update the image
        service['image'] = hub_image   

In [ ]:
# Reset the stack services to the new updated ones
#stack = yaml.load(open(input_file))
stack['services'] = services

In [ ]:
######################## Build Compose File ###################################
for service_list in config['services']:
    for key, value in service_list.items():
        temp_stack = {}
        temp_stack['services'] = {}
        temp_stack['version'] = stack['version']
        
        # Add services
        for service in value:
            temp_stack['services'][service] = stack['services'][service]
           # Check depend on. If not in list, then need to remove
            try:
                if stack['services'][service]['depends_on'] not in value:
                    del temp_stack['services'][service]['depends_on']
            except:
                pass
                    
        # output yaml file using key in filename
        output_file = '{}_docker_compose.yml'.format(key)
        print('Creating docker file: {}'.format(output_file))
        with open(output_file, "w") as f:
            yaml.safe_dump(temp_stack, f, default_flow_style=False)
        # yaml that is produced is a bit buggy.
        fh = open(output_file, "r+")
        lines = map(lambda a: re.sub(r"^\s{4}-", "      -", a), fh.readlines())
        fh.close()
        with open(output_file, "w") as f:
            f.writelines(lines)
        print('File Built Successfully')